Построить классификатор, который "угадывает" персонажа по его фразе. Baseline: равновероятный выбор между всеми главными героями.
Примерный план действий: выделить реплики главных персонажей, сгруппировать их по героям, нормализовать (свой выбор процедур объяснить (например, нужно ли включать в стоп-лист обсценную лексику?); из инструментов для английского удобно использовать nltk), векторизовать (например, CountVectorizer), отрезать поверочную выборку (эту часть не трогаем до последней проверки). На оставшейся, большей, части данных: 1) немного поанализировать данные, чтобы понять, какие признаки могут помочь при обучении — например, построить частотные списки и сравнить их (достаточно ли матрицы терм-документ с ненормализованными вхождениями слов?);  2) обучить модели (лес, наивный байес, логит) и подобрать их оптимальные параметры. Интерпретировать параметры (атрибуты) лучших моделей. Протестировать их на проверочной выборке, выбрать лучшую модель, проиллюстрировать результат (например, нарисовать decision surface). Сравнить результат с нехитрым Baseline классификатором.

In [1]:
#imports
import pandas as pd


In [77]:
#how raw daata looks like
raw_data = pd.read_csv('./SouthParkData/All-seasons.csv')
raw_data.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


In [78]:
#little summary, not all lines are unique. Moreover alot of characters
raw_data.describe()

,Season,Episode,Character,Line
count,70896,70896,70896,70896
unique,19,19,3950,64301
top,2,10,Cartman,What?\n
freq,6416,5271,9774,361


In [79]:
#count lines for character. Data have big "tale" of more or less unique characters
#It seems reasonable to take only those caracters, who spoke at least 100 times (sligtly more than once per episode)
raw_data.groupby(['Character']).size().sort_values(ascending=False)

Character
Cartman               9774
Stan                  7680
Kyle                  7099
Butters               2602
Randy                 2467
Mr. Garrison          1002
Chef                   917
Kenny                  881
Sharon                 862
Mr. Mackey             633
Gerald                 626
Jimmy                  597
Wendy                  585
Liane                  582
Sheila                 566
Jimbo                  556
Announcer              407
Stephen                357
Craig                  326
Clyde                  317
Jesus                  312
Linda                  290
Principal Victoria     289
Terrance               282
Mrs. Garrison          282
Token                  278
Timmy                  263
Mayor                  245
Tweek                  233
Phillip                222
                      ... 
M.C                      1
MJ's Hologram            1
Ma'am                    1
Madonna                  1
Magician                 1
Male Activist     

In [230]:
top_speakers = raw_data.groupby(['Character']).size(
    ).loc[raw_data.groupby(['Character']).size() > 2000]
#print(top_speakers.index.values)
main_char_lines = raw_data.loc[raw_data['Character'].isin(top_speakers.index.values)]
main_char_lines.describe()
#main_char_lines

,Season,Episode,Character,Line
count,29622,29622,29622,29622
unique,18,18,5,27053
top,6,8,Cartman,What?\n
freq,2422,2245,9774,207


In [227]:
from sklearn.model_selection import train_test_split

main_char_lines['Line'] = [line.replace('\n','') for line in main_char_lines['Line']]
train, test = train_test_split(main_char_lines, test_size=0.3, random_state=14)

/Users/Ksenia/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [228]:
#preprocess data
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
from nltk.stem.lancaster import LancasterStemmer

st = LancasterStemmer()
def token(text):
    txt = nltk.word_tokenize(text.lower())
    return [st.stem(word) for word in txt]


stop = set(stopwords.words("english"))
cv = CountVectorizer(#lowercase=True, 
                     tokenizer=token, #stop_words=stop,# token_pattern=u'(?u)\b\w\w+\b',
                     analyzer=u'word', min_df=4)
#print(train['Line'].tolist())

vec_train = cv.fit_transform(train['Line'].tolist())
vec_test = cv.transform(test['Line'].tolist())

#print(vec_train)

In [219]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score

nb = MultinomialNB()
print()
nb.fit(vec_train, train['Character'])

accuracy_score(nb.predict(vec_test), test['Character'])

0.47858107278752915

In [229]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()#multi_class='multinomial')
lr.fit(X = vec_train, y = train['Character'])

accuracy_score(lr.predict(vec_test), test['Character'])

0.46584899291099358

In [221]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(X = vec_train, y = train['Character'])

accuracy_score(rf.predict(vec_test), test['Character'])

0.47723088253344792

In [222]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, vec_train, train['Character'])
print(scores.mean())
clf.fit(vec_train, train['Character'])
print(accuracy_score(clf.predict(vec_test), test['Character']))

0.457017640262
0.462624278876
